In [1]:
from ltlcross_runner import LtlcrossRunner
from IPython.display import display, display_markdown
import pandas as pd

If `rerun` is set to `False` the script uses the precomputed values. If set to `True`, all values are recomputed
(takes more time)

In [2]:
### Tools' setting ###
# PATHS
owl_version = '18.06'
owl_bin     = 'owl-'+owl_version+'/bin/'
make_tgba   = 'ltl2tgba --deterministic -f %f' 
# Tools
owl         = owl_bin+'ltl2ldgba -i %f -n'
nba2ldba    = make_tgba + ' --ba | '+owl_bin+'nba2ldba'
seminator   = make_tgba + ' | ./seminator'
sem_enter   = make_tgba + ' | ./seminator --jump-enter'
sem_always  = make_tgba + ' | ./seminator --jump-always'
cy          = make_tgba + ' | ./seminator --via-tba'
# Options
cut         = ' --cd'
simp_sd     = ' | autfilt --small --tgba'
simp_cd     = ' | autfilt -x simul=1,ba-simul=1 --small --tgba'
nos         = ' -s0' # disables Spot's simplifications used in Seminator
end         = ' > %O' # saves result to file

### Ltlcross runner configuration ###
tools = {## Cut-deterministic
         'cd.ltl2ldba.no'     : owl + end,
         'cd.ltl2ldba.yes'    : owl + simp_cd + end,
         'cd.seminator.no'    : seminator + nos + cut + end,
         'cd.seminator.yes'   : seminator + cut + end,
         'cd.sem_enter.no'    : sem_enter + nos + cut + end,
         'cd.sem_enter.yes'   : sem_enter + cut + end,
         'cd.sem_always.no'   : sem_always + nos + cut + end,
         'cd.sem_always.yes'  : sem_always + cut + end,
         'cd.2-step.no'       : cy + nos + cut + end,     
         'cd.2-step.yes'      : cy + cut + end,
         ### Semi-deterministic
         'sd.ltl2ldba.no'     : owl + ' -n' + end,
         'sd.ltl2ldba.yes'    : owl + ' -n' + simp_sd + end,
         'sd.nba2ldba.no'     : nba2ldba + end,
         'sd.nba2ldba.yes'    : nba2ldba + simp_sd + end,
         'sd.seminator.no'    : seminator + nos + end,
         'sd.seminator.yes'   : seminator + end,
         'sd.sem_enter.no'    : sem_enter + nos + end,
         'sd.sem_enter.yes'   : sem_enter + end,
         'sd.sem_always.no'   : sem_always + nos + end,
         'sd.sem_always.yes'  : sem_always + end,
         'sd.2-step.no'       : cy + nos + end,  
         'sd.2-step.yes'      : cy + end,
        }
sd_tools = ['sd.seminator.no','sd.seminator.yes',
            'sd.sem_enter.no','sd.sem_enter.yes',
            'sd.sem_always.no','sd.sem_always.yes',
            'sd.2-step.no','sd.2-step.yes',
            'sd.ltl2ldba.no','sd.ltl2ldba.yes',
            'sd.nba2ldba.no','sd.nba2ldba.yes'
           ]
cd_tools = ['cd.seminator.no','cd.seminator.yes',
            'cd.sem_enter.no','cd.sem_enter.yes',
            'cd.sem_always.no','cd.sem_always.yes',
            'cd.2-step.no','cd.2-step.yes',
            'cd.ltl2ldba.no','cd.ltl2ldba.yes'
           ]
### Numbers to measure ###
cols = ['states','edges','transitions','nondet_states','exit_status','time','acc']
tool_order = ['seminator','sem_enter','sem_always','2-step','ltl2ldba','nba2ldba']

In [3]:
rerun = True

In [4]:
runners = {}
for source in ('literature','random'):
    for t in ('det','cd','sd','nd'):
        name = '{}_{}'.format(source,t)
        runners[name] = \
            LtlcrossRunner(tools,\
                    res_filename='data/{}.csv'.format(name),\
                    formula_files=['formulae/{}.ltl'.format(name)],\
                    cols=cols)
        if rerun:
            runners[name].run_ltlcross(timeout='120',check=False)
        runners[name].parse_results()

In [5]:
def get_counts(runner):
    v = r.values.states.copy()
    v.columns = pd.MultiIndex.from_tuples([tuple(c.split('.')) for c in v.columns])
    return v.cd.dropna().count()[0], v.sd.dropna().count()[0]

## Cummulative results

Gather the data into one big DataFrame

In [6]:
data = pd.DataFrame()
for (name,r) in runners.items():
    # Divide into sd_tools & cd_tools to treat timeouts separately.
    tmp_cd = pd.DataFrame(r.cummulative(tools=cd_tools),columns=[name])
    tmp_sd = pd.DataFrame(r.cummulative(tools=sd_tools),columns=[name])
    data = data.append(tmp_cd.append(tmp_sd).transpose())
data

tool,cd.seminator.no,cd.seminator.yes,cd.sem_enter.no,cd.sem_enter.yes,cd.sem_always.no,cd.sem_always.yes,cd.2-step.no,cd.2-step.yes,cd.ltl2ldba.no,cd.ltl2ldba.yes,...,sd.sem_enter.no,sd.sem_enter.yes,sd.sem_always.no,sd.sem_always.yes,sd.2-step.no,sd.2-step.yes,sd.ltl2ldba.no,sd.ltl2ldba.yes,sd.nba2ldba.no,sd.nba2ldba.yes
literature_det,556,556,556,556,556,556,585,585,1971,773,...,556,556,556,556,585,585,1971,730,600,600
literature_cd,194,194,194,194,194,194,198,198,517,331,...,194,194,194,194,198,198,517,285,207,207
literature_sd,13,13,13,13,13,13,13,13,28,19,...,13,13,13,13,13,13,28,16,13,13
literature_nd,622,397,622,385,633,399,644,434,453,354,...,444,321,455,319,468,388,455,327,644,411
random_det,415,415,415,415,415,415,416,416,514,460,...,415,415,415,415,416,416,514,428,428,428
random_cd,464,464,464,464,464,464,464,464,688,588,...,464,464,464,464,464,464,688,545,506,506
random_sd,734,712,734,712,734,712,735,713,921,772,...,704,704,704,704,705,705,921,688,720,720
random_nd,2019,1122,2033,978,2053,998,2093,1176,1084,833,...,1223,759,1247,749,1251,970,1083,750,1643,1003


Get the number of formulas such that all tools finished within timeout for each category.

In [7]:
counts = pd.DataFrame(index=pd.Index([],name='source'),columns=['cd..n','sd..n'])
for (name,r) in runners.items():
    cd_c, sd_c = get_counts(r)
    counts = counts.append(pd.DataFrame({'cd..n':cd_c,'sd..n':sd_c},index=[name]))
counts

,cd..n,sd..n
literature_det,149,149
literature_cd,46,46
literature_sd,3,3
literature_nd,23,23
random_det,100,100
random_cd,100,100
random_sd,100,100
random_nd,100,100


Converts the DataFrame to use MultiIndices for rows and columns

In [8]:
from pandas.api.types import CategoricalDtype
automata_type = CategoricalDtype(categories=['det','cd','sd','nd'], ordered=True)
source_type = CategoricalDtype(categories=['random', 'literature', 'rand. (T/O)', 'lit. (T/O)'], ordered=True)

In [9]:
def make_hierarchical(data):
    # Split index into origin & ltl2tgba's output type
    df = data.copy()
    df.index.name = 'origin'
    df.reset_index(inplace=True)
    df['type'] = df.origin.apply(lambda x: x.split('_')[1]).astype(automata_type)
    df['origin'] = df.origin.apply(lambda x: x.split('_')[0]).astype(source_type)
    df = df.set_index(['origin','type']).sort_index()
    # Split columns in tool & reductions
    df.columns = pd.MultiIndex.from_tuples([tuple(c.split('.')) for c in df.columns])
    return df

In [10]:
tmp = make_hierarchical(data.join(counts)).sort_index(axis=1)
tmp

cd                                                       \
                     2-step       ltl2ldba      sem_always      sem_enter   
                   n     no   yes       no  yes         no  yes        no   
origin     type                                                             
random     det   100    416   416      514  460        415  415       415   
           cd    100    464   464      688  588        464  464       464   
           sd    100    735   713      921  772        734  712       734   
           nd    100   2093  1176     1084  833       2053  998      2033   
literature det   149    585   585     1971  773        556  556       556   
           cd     46    198   198      517  331        194  194       194   
           sd      3     13    13       28   19         13   13        13   
           nd     23    644   434      453  354        633  399       622   

                               ...        sd                                 \
                     seminator ...  ltl2ldba      nba2ldba       sem_always   
                 yes        no ...        no  yes       no   yes         no   
origin     type                ...                                            
random     det   415       415 ...       514  428      428   428        415   
           cd    464       464 ...       688  545      506   506        464   
           sd    712       734 ...       921  688      720   720        704   
           nd    978      2019 ...      1083  750     1643  1003       1247   
literature det   556       556 ...      1971  730      600   600        556   
           cd    194       194 ...       517  285      207   207        194   
           sd     13        13 ...        28   16       13    13         13   
           nd    385       622 ...       455  327      644   411        455   

                                                    
                     sem_enter      seminator       
                 yes        no  yes        no  yes  
origin     type                                     
random     det   415       415  415       415  415  
           cd    464       464  464       464  464  
           sd    704       704  704       704  704  
           nd    749      1223  759      1211  921  
literature det   556       556  556       556  556  
           cd    194       194  194       194  194  
           sd     13        13   13        13   13  
           nd    319       444  321       444  353  

[8 rows x 24 columns]

In [11]:
cd = tmp.cd[['']+tool_order]
sd = tmp.sd[['']+tool_order]

In [12]:
cd

seminator       sem_enter      sem_always      2-step  \
                   n        no   yes        no  yes         no  yes     no   
origin     type                                                              
random     det   100       415   415       415  415        415  415    416   
           cd    100       464   464       464  464        464  464    464   
           sd    100       734   712       734  712        734  712    735   
           nd    100      2019  1122      2033  978       2053  998   2093   
literature det   149       556   556       556  556        556  556    585   
           cd     46       194   194       194  194        194  194    198   
           sd      3        13    13        13   13         13   13     13   
           nd     23       622   397       622  385        633  399    644   

                      ltl2ldba       
                  yes       no  yes  
origin     type                      
random     det    416      514  460  
           cd     464      688  588  
           sd     713      921  772  
           nd    1176     1084  833  
literature det    585     1971  773  
           cd     198      517  331  
           sd      13       28   19  
           nd     434      453  354

In [13]:
sd

seminator      sem_enter      sem_always      2-step  \
                   n        no  yes        no  yes         no  yes     no   
origin     type                                                             
random     det   100       415  415       415  415        415  415    416   
           cd    100       464  464       464  464        464  464    464   
           sd    100       704  704       704  704        704  704    705   
           nd    100      1211  921      1223  759       1247  749   1251   
literature det   149       556  556       556  556        556  556    585   
           cd     46       194  194       194  194        194  194    198   
           sd      3        13   13        13   13         13   13     13   
           nd     23       444  353       444  321        455  319    468   

                     ltl2ldba      nba2ldba        
                 yes       no  yes       no   yes  
origin     type                                    
random     det   416      514  428      428   428  
           cd    464      688  545      506   506  
           sd    705      921  688      720   720  
           nd    970     1083  750     1643  1003  
literature det   585     1971  730      600   600  
           cd    198      517  285      207   207  
           sd     13       28   16       13    13  
           nd    388      455  327      644   411

## Timeouts

In [14]:
TO_data = pd.DataFrame()
for (name,r) in runners.items():
    TO_data = TO_data.append(pd.DataFrame(r.compute_timeouts(),columns=[name]).transpose())
TO = make_hierarchical(TO_data).replace(0,pd.NaT).dropna(how='all').fillna(0)
TO

Empty DataFrame
Columns: [(cd, 2-step, no), (cd, 2-step, yes), (cd, ltl2ldba, no), (cd, ltl2ldba, yes), (cd, sem_always, no), (cd, sem_always, yes), (cd, sem_enter, no), (cd, sem_enter, yes), (cd, seminator, no), (cd, seminator, yes), (sd, 2-step, no), (sd, 2-step, yes), (sd, ltl2ldba, no), (sd, ltl2ldba, yes), (sd, nba2ldba, no), (sd, nba2ldba, yes), (sd, sem_always, no), (sd, sem_always, yes), (sd, sem_enter, no), (sd, sem_enter, yes), (sd, seminator, no), (sd, seminator, yes)]
Index: []

[0 rows x 22 columns]

There is no timeout.

# Final results
We use the `display` function to propagate the results to notebooks that just run this one.

In [15]:
display_markdown('''### Comparison of tools producing cut-deterministic automata
All tools finished within the one-minute time limit.
''',raw=True)

### Comparison of tools producing cut-deterministic automata
All tools finished within the one-minute time limit.


In [16]:
display(cd)

seminator       sem_enter      sem_always      2-step  \
                   n        no   yes        no  yes         no  yes     no   
origin     type                                                              
random     det   100       415   415       415  415        415  415    416   
           cd    100       464   464       464  464        464  464    464   
           sd    100       734   712       734  712        734  712    735   
           nd    100      2019  1122      2033  978       2053  998   2093   
literature det   149       556   556       556  556        556  556    585   
           cd     46       194   194       194  194        194  194    198   
           sd      3        13    13        13   13         13   13     13   
           nd     23       622   397       622  385        633  399    644   

                      ltl2ldba       
                  yes       no  yes  
origin     type                      
random     det    416      514  460  
           cd     464      688  588  
           sd     713      921  772  
           nd    1176     1084  833  
literature det    585     1971  773  
           cd     198      517  331  
           sd      13       28   19  
           nd     434      453  354

In [17]:
display_markdown('''### Comparison of tools producing semi-deterministic automata
All tools finished within the one-minute time limit.
''',raw=True)

### Comparison of tools producing semi-deterministic automata
All tools finished within the one-minute time limit.


In [18]:
display(sd)

seminator      sem_enter      sem_always      2-step  \
                   n        no  yes        no  yes         no  yes     no   
origin     type                                                             
random     det   100       415  415       415  415        415  415    416   
           cd    100       464  464       464  464        464  464    464   
           sd    100       704  704       704  704        704  704    705   
           nd    100      1211  921      1223  759       1247  749   1251   
literature det   149       556  556       556  556        556  556    585   
           cd     46       194  194       194  194        194  194    198   
           sd      3        13   13        13   13         13   13     13   
           nd     23       444  353       444  321        455  319    468   

                     ltl2ldba      nba2ldba        
                 yes       no  yes       no   yes  
origin     type                                    
random     det   416      514  428      428   428  
           cd    464      688  545      506   506  
           sd    705      921  688      720   720  
           nd    970     1083  750     1643  1003  
literature det   585     1971  730      600   600  
           cd    198      517  285      207   207  
           sd     13       28   16       13    13  
           nd    388      455  327      644   411

In [19]:
runners['random_nd'].smaller_than('cd.ltl2ldba.yes','cd.sem_enter.yes')

,tool,cd.ltl2ldba.yes,cd.sem_enter.yes
form_id,formula,,
1,G(Fa U X(b & Fc)),6,41
2,FG(Ga U X(Ga R b)),5,7
4,G((((a & b) | (!a & !b)) & (GF!b U !c)) | (((!a & b) | (a & !b)) & (FGb R c))),9,13
7,FG(Fa U (Fb R c)),4,37
8,GF(a & Xb) | (c & Ga),4,7
10,G(a U Xb),2,5
11,XXG(Fa | Gb),6,7
16,GFa R b,3,5
17,GFa | G(Gb | Xa),4,7


In [21]:
runners['random_nd'].smaller_than('cd.sem_always.yes','cd.sem_enter.yes')

,tool,cd.sem_always.yes,cd.sem_enter.yes
form_id,formula,,
2,FG(Ga U X(Ga R b)),6,7
3,XFa U G(Gc | Fb),5,6
8,GF(a & Xb) | (c & Ga),6,7
10,G(a U Xb),4,5
16,GFa R b,4,5
17,GFa | G(Gb | Xa),6,7
20,G(a | X(GFa | (a M b))) R !b,12,15
22,X(a & (X((G!b & XGc) | (Fb & XF!c)) R Fc)),13,14
26,Ga | (Ga R Fb),4,5
